In [27]:
import numpy as np
import pandas as pd
import time

import serial
import serial.tools.list_ports

from pathlib import Path

# import workbench.config.config'
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

In [28]:
ports = serial.tools.list_ports.comports()
ports

In [29]:
[port.manufacturer for port in ports]

['Microsoft']

In [31]:
port = ports[0].device

In [32]:
# Open port
arduino = serial.Serial(port, baudrate=115200, timeout=1)

In [33]:
def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=000
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout


In [34]:
# Call the handshake function
handshake_arduino(arduino, print_handshake_message=True)

Handshake message: Start



In [35]:
def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    line = raw.rstrip()

    return line

It is important to always close the ports when they are not used!

In [38]:
def get_arduino_lines(n_data = 1000, port="COM3"):
    arduino = serial.Serial(port, baudrate=115200, timeout=1)

    handshake_arduino(arduino, print_handshake_message=True)
    
    # arduino.close()
    # arduino.open()

    lines =[]
    i = 0

    while i < n_data:
        try:
            raw = arduino.read_until()
            raw = parse_raw(raw)
            lines.append(raw)
            i +=1
        except:
            "Print"
    arduino.close()

    return lines

In [45]:
arduino.close()

In [46]:
scan_lines = get_arduino_lines()

Handshake message: Inference time 



# Cleanup the received data

In [119]:
def create_MCU_inference_df(lines):
    clean_lines = []
    for line in lines:
        clean_lines.append(str(line).replace("lu", ""))#.replace(").", ""))

    #clean_lines
    lines_df = pd.DataFrame(clean_lines)

    split_df = lines_df[0].str.split(",",expand=True)
    # setting first row as headers
    split_df.columns = split_df.iloc[0]
    split_df = split_df[1:]
    split_df.columns = [c.strip('"') for c in split_df.columns.values.tolist()]

    # data cleaning
    split_df = split_df[split_df['Ticks'] != '"Ticks"']
    split_df = split_df.fillna(0)
    split_df["Ticks"] = split_df["Ticks"].astype(int)
    split_df["ms"] = split_df["Ticks"]/ 1000.0

    # finding start and end of each inference
    # each inference starts and ends with quantization of the input or output tensor
    split_df["next_Tag"] = split_df["Tag"].shift()
    split_df["prev_Tag"] = split_df["Tag"].shift(-1)
    split_df["start"] = (split_df["next_Tag"] == split_df["Tag"])
    split_df["end"] = (split_df["prev_Tag"] == split_df["Tag"])

    # create markers for the different inference cycles
    inference = []
    i =1 
    for row in split_df['start']:
        if row == True:
            inference.append(f"inference_{i}")
            i = i+1
        else:
            inference.append(None)

    split_df['inference'] = inference
    split_df['inference'].fillna(method="ffill", inplace=True)
    
    split_df.drop(["start", "end","next_Tag", "prev_Tag"], axis=1, inplace=True)

    return split_df

    


In [120]:
inference_df = create_MCU_inference_df(scan_lines)
inference_df

,Event,Tag,Ticks,ms,inference
1,0,QUANTIZE,5785,5.785,None
2,1,CONV_2D,69562,69.562,None
3,2,MAX_POOL_2D,6821,6.821,None
4,3,CONV_2D,8157,8.157,None
5,4,DEPTHWISE_CONV_2D,1356,1.356,None
...,...,...,...,...,...
995,19,CONV_2D,2175,2.175,inference_6
996,20,DEPTHWISE_CONV_2D,1789,1.789,inference_6
997,21,CONV_2D,2175,2.175,inference_6
998,22,CONCATENATION,108,0.108,inference_6


In [122]:
inference_1_df = inference_df[inference_df["inference"]== "inference_1"]
inference_1_df

,Event,Tag,Ticks,ms,inference
140,139,QUANTIZE,5885,5.885,inference_1
141,140,CONV_2D,70083,70.083,inference_1
142,141,MAX_POOL_2D,6915,6.915,inference_1
143,142,CONV_2D,8293,8.293,inference_1
144,143,DEPTHWISE_CONV_2D,1372,1.372,inference_1
...,...,...,...,...,...
274,273,CONV_2D,61694,61.694,inference_1
275,274,MEAN,48612,48.612,inference_1
276,275,FULLY_CONNECTED,302,0.302,inference_1
277,276,SOFTMAX,55,0.055,inference_1


In [123]:
inference_time = inference_1_df["ms"].sum()
inference_time

363.009